In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#json
import json
import matplotlib.pyplot as plt
import requests
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import nltk
from nltk.corpus import stopwords
from tqdm import tqdm
from collections import Counter
import re

In [ ]:
def create_taggedDocument_from_json(dataInd,fileId):
    
    filename = "../input/coleridgeinitiative-show-us-the-data/" + dataInd + "/" + fileId + ".json"
    
    fd = open(filename, mode='r')
    data = json.load(fd)
    fd.close()
    json_text = json.dumps(data).replace('\\n',' ').replace('\\f',' ').replace('\\u','!!!').replace('\\b',' ').replace('\\t',' ').replace('\\',' ')
    json_text = re.sub('!{3}[A-Za-z0-9]{4}',' ',json_text)
    textWordlist = nltk.word_tokenize(json_text)

    #STOPWORD
    #return TaggedDocument(words=textWordlist, tags=[fileId])

    #STOPWORD
    stopWords = stopwords.words('english') + \
   ['"','}', 'then', 'result', 'been', '%', 'to', 'due', 'important',
    'one', 'However', 'other', 'first', 'be', '``', 'for', 'with', ']', 'This', 'not', 'These', '{', 
    'because', 'The', 'at', 'did', 'that', 'research', 'high', 'of', 'than', 'related', 'into', 'when', 
     ':', '5', 'were', 'may', 'those', ')', 'As', 'an', 'data', 'all', 'over', 'these', 'the', 'this', 
     'level', 'will', 'based', 'less', 'can', 'text', 'a', 'some', 'different', 'large', 'included',
     'available', 'who', '(', ',', 'within', 'two', "'s", 'differences', 'and', 'similar', 'have', '1', 
     'it', 'through', 'each', 'three', 'analysis', 'as', 'total', 'significant', 'about', 'while', 'is', 
     'on', 'but', 'if', '[', 'In', 'reported', 'Table', 'in', 'among', 'likely', 'which', 'studies', 
     'section_title', 'by', 'both', 'between', "''", 'across', 'using', 'more', 'information', 'A', 
     'also', 'addition', 'study', ';', '.', 'or', 'any', 'are', 'has', 'had', 'For', 'results', 'used', 
     'from', 'was', 'time', 'associated', '20', 'could', 'after', 'number', 'found', 'such', '9', '3', 
     'lower', '4', 'would', 'many', '8', 'further', '7', '6', 'years', 'higher', 'shown', 'et', '2', 
     'several', 'compared', 'they', 'their', 'use', 'To', 'most', 'during', 'same', 'including', 
     'model', 'following', 'no', 'small', 'do', 'only', 'we', 'should', 'well', '12', 'include', 
     'possible', '10', 'We', 'where', 'there', 'increase', 'example', 'It', 'provide', 'our', 
     'potential', 'its']
    
    wordlist = [word for word in textWordlist if word.lower() not in stopWords]
    return TaggedDocument(words=wordlist, tags=[fileId])

In [ ]:
sample_submission_df = pd.read_csv("../input/coleridgeinitiative-show-us-the-data/sample_submission.csv")
train_df = pd.read_csv("../input/coleridgeinitiative-show-us-the-data/train.csv")

In [ ]:
training_docs = []

distinct_train_df = train_df.drop_duplicates(subset=["Id"])

for Id in distinct_train_df["Id"]:
    training_docs.append(create_taggedDocument_from_json("train", Id))

for Id in sample_submission_df["Id"]:
    training_docs.append(create_taggedDocument_from_json("test", Id))

model = Doc2Vec(documents=training_docs, min_count=1, dm=0)
 
    # print(model.docvecs['d1'])

In [ ]:
predict_df = pd.DataFrame([], columns = ['test_Id', 'cleaned_label'])

for Id in sample_submission_df["Id"]:
    result_df = pd.DataFrame([], columns = ['test_Id' , 'train_Id', 'train_cos', 'cleaned_label'])
    doccnt = 0
    loc = 0
    for doc in model.docvecs.most_similar(Id, topn=1):
        queryString = 'Id == "' + doc[0] + '"'
        temp_df = train_df.query(queryString)
        if len(temp_df) > 0:
            doccnt = doccnt + 1
            for index, row in temp_df.iterrows():
                result_df.loc[loc] = [Id, doc[0], doc[1], row['cleaned_label']]
                loc = loc + 1
    
    result_df = result_df.groupby(['test_Id', 'cleaned_label'], as_index=False).count()
    result_df = result_df.sort_values(['test_Id', 'train_Id'], ascending=[True, False])
    result_df = result_df.groupby('test_Id')['cleaned_label'].apply('|'.join).reset_index()
    predict_df = predict_df.append(result_df)

submit_df = pd.merge(sample_submission_df, predict_df, how='left', left_on='Id', right_on='test_Id')
submit_df = submit_df.fillna("")
submit_df.head()

In [ ]:
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
import sys

In [ ]:
my_submission = pd.DataFrame({'Id': submit_df.Id, 'PredictionString': submit_df.cleaned_label})
# you could use any filename. We choose submission here
my_submission.to_csv('submission.csv', index=False)